```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'num'/... - определяем через isinstance()
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        или пар (тип, слово), где тип - main/punct/other
```

```
str(древовидная структура)
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

# Общее

In [1]:
def throw(ex):
    raise ex

In [2]:
class TextError(ValueError):
    pass
class ParseError(ValueError):
    pass
class TestError(ValueError):
    pass


# Паттерны парсинга

In [3]:
from copy import deepcopy,copy
import re

In [4]:
PUNCT_CHARS=".,:;?!'-"

In [5]:
class SAttrs:
    __slots__=['pre','changers','tags']
    def __init__(self,pre='',changers=None,tags=None):
        if changers==None: changers=set()
        if tags==None: tags=set()
        self.pre=pre
        self.changers=changers
        self.tags=tags
    def change(self,s):
        for changer in self.changers:
            s=changer(s)
        # todo gtags
        return self.pre+s
    def __repr__(self):
        return 'SAttrs<'+str(id(self))+'>(pre='+repr(self.pre)+\
            ',changers='+(('{'+
                ','.join('<'+i.__name__+'>' for i in self.changers)+'}') \
                    if len(self.changers)>0 else 'set()')+\
            ',tags='+repr(self.tags)+')'
    
    @staticmethod
    def join(arr):
        def subr(g):
            yield str(next(g))
            for i in g:
                s=str(i)
                if i.attrs.pre!='' or re.fullmatch('['+re.escape(PUNCT_CHARS)+']*',s):
                    yield s
                else: yield ' '+s
        return ''.join(subr(iter(arr)))

    @staticmethod
    def to_right(l,r):
        if r.attrs.pre=='':
            r.attrs.pre=l.attrs.pre
        r.attrs.changers|=l.attrs.changers
        r.attrs.tags|=l.attrs.tags
        return r
    @staticmethod
    def to_left(l,r):
        l.attrs.change|=r.attrs.change
        l.attrs.tags|=r.attrs.tags
        return l


In [6]:
class S(str): # строка с атрибутом
    __slots__='attrs'
    def __new__(cls,s,attrs=None):
        return str.__new__(cls,s)
    def __init__(self,s,attrs=None):
        self.attrs = attrs if attrs!=None else SAttrs()
        
    def __repr__(self):
        return 'S('+str.__repr__(self)+','+repr(self.attrs)+')'
    def __str__(self):
        return self.attrs.change(str.__str__(self))

#.pre - строка - вместо начальных пробелов
#.tags - множество пар строк - объединяется
#.changers - множество функций - объединяется

In [7]:
def sp_seq(str,pos,patterns):
#    for patt in patterns:
#        tmp=patt(str,pos)
#        if len(tmp)>1 : raise NotImplementedError()
#        if len(tmp)==0 : return []
#        (pos,tmp)=tmp[0]
#        rezs.append(tmp)
#    return [(pos,rezs)]
    if len(patterns)==1:
        return [(p,[r]) for (p,r) in patterns[0](str,pos)]
    first=patterns[0](str,pos)
    first.sort(key=lambda i:i[0]) # в дальнейшем отключить повторное вычисление 
    # продолжения для одинаковых позиций
    rezs=[]
    for r in first:
        tmp=sp_seq(str,r[0],patterns[1:])
        for rr in tmp:
            rr[1].insert(0,r[1])
        rezs+=tmp
    return rezs

In [8]:
# 'word'
def sp_const_word(str,pos,word):
    return [(pos+len(word),word)] if str[pos:pos+len(word)]==word else []


In [9]:
def ch_title(s):
    return s.title()
def ch_upper(s):
    return s.upper()
def ch_sentence(s):
    if len(s)==0: return ''
    return s[0].upper()+s[1:]

# [a-zA-Z]+
def sp_word(str,pos):
    pos1=pos
    while pos1<len(str) and \
        (str[pos1]>='a' and str[pos1]<='z' or str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    if pos1==pos:
        return []

    if str[pos:pos1]=='I':
        return [(pos1,S('I'))]

    s=S(str[pos:pos1].lower())
    if str[pos:pos1].islower():
        pass
    elif str[pos:pos1].istitle():
        s.attrs.changers={ch_title}
    elif str[pos:pos1].isupper():
        s.attrs.changers={ch_upper}
    else:
        print(s,' - перепутаны заглавные и малые буквы')
    return [(pos1,s)]

In [10]:
def sp_punct(str,pos):
    pos1=pos
    while pos1<len(str) and str[pos1] in PUNCT_CHARS :
        pos1+=1
    return [] if pos1==pos else [(pos1,S(str[pos:pos1]))]

In [11]:
def sp_open_tag(s,p):
    return []
def sp_close_tag(s,p):
    return []
def sp_openclose_tag(s,p):
    return sp_const_word(s,p,'<br>')

In [12]:
# ([ _\r\n\v\t]|sp_openclose_tag)+
def sp_spcs(str,pos):
    pre=''
    pos1=pos
    while pos1<len(str):
        if str[pos1] in ' _\t\n\r\v':
            pre+=str[pos1]
            pos1+=1
            continue
        tmp = sp_openclose_tag(str,pos1)
        if len(tmp)!=0:
            pre+=tmp[0][1]
            pos1=tmp[0][0]
            continue
        break
    return [] if pos1==pos else [(pos1,pre)]    

In [13]:
def tokenizer(s):
    pos=0
    pre=''
    while pos<len(s):
        tmp=sp_spcs(s,pos)
        if len(tmp)>0:
            (pos,pre)=tmp[0]
            if pre==' ': pre=''
            continue
        tmp=sp_word(s,pos)
        if len(tmp)>0:
            (pos,foryield) = tmp[0]
            foryield.attrs.pre=pre; pre=''
            yield foryield
            continue
        tmp=sp_punct(s,pos)
        if len(tmp)>0:
            (pos,foryield) = tmp[0]
            foryield.attrs.pre=pre; pre=''
            yield foryield
            continue
        raise TextError("can't tokenize: "+\
                        repr(s[max(0,pos-10):pos])+' - '+repr(s[pos:min(len(s),pos+10)]))


In [14]:
def tokenize(s) : return [i for i in tokenizer(s)]

In [15]:
#объекты из словаря и паттернов копируются (полностью), потом из них строится дерево

#в эти копии потом добваляется .attrs
def word(w):
    assert type(w)==str
    return lambda s,p: \
        [(p+1,deepcopy(s[p]))] if p<len(s) and s[p]==w else []

def fromdict(d):
    assert type(d)==dict
    def p_fromdict(s,p):
        if p<len(s) and s[p] in d:
            tmp=deepcopy(d[s[p]])
            tmp.attrs=deepcopy(s[p].attrs)
            return [(p+1,tmp)]
        else:
            return []
    return p_fromdict

def p_seq(s,p,patterns,handler,numbers=None):
# p_seq(s,p,[p0,p1,p2,p3],handle,[0,2,3]) ->
# for k:
#   (pos1,rez1) = sp_seq(s,p,[p0,p1,p2,p3]) [k]
#   rezs.append((pos1,handle(rezs[0],rezs[2],rezs[3],)))
    if numbers==None : numbers=range(len(patterns))
    rezs=[]
    for (pos1,rez1) in sp_seq(s,p,patterns):
        m=[False for i in range(len(patterns))]
        for i in numbers: m[i]=True
        for i in range(max(numbers)):
            if not m[i]: SAttrs.to_right(rez1[i],rez1[i+1])
        for i in range(len(patterns)-1,max(numbers),-1):
            if not m[i]: SAttrs.to_left(rez1[i-1],rez1[i])
        rez2=handler(*[rez1[i] for i in numbers])
#        for i in rez1:
#            if isinstance(i,StDeclinable):
#                i.check_attrs('p_seq')
#        if isinstance(rez2,StDeclinable):
#                rez2.check_attrs('p_seq:'+handler.__name__)
        rezs.append((pos1,rez2))
    return rezs

##### тесты

In [16]:
SAttrs.join([S('a'),S(',')])

'a,'

In [17]:
x=[S('a') for i in range(5)]
[id(i) for i in x]

[86424936, 86425048, 86425160, 86425272, 86425384]

In [18]:
sp_word('a',0)

[(1, S('a',SAttrs<86344136>(pre='',changers=set(),tags=set())))]

In [19]:
print(repr(tokenize('a')))

[S('a',SAttrs<86274824>(pre='',changers=set(),tags=set()))]


In [20]:
x=S('qwer')
x.attrs.pre='pre'
x.attrs.changers={ch_title}
str(x)

'preQwer'

In [21]:
x=S('qwe')
x.attrs.pre='asd'
repr(x)

"S('qwe',SAttrs<83857224>(pre='asd',changers=set(),tags=set()))"

In [22]:
def h(*args):
    print(args)
p_seq([S('1'),S('2'),S('3')],0,[word('1'),word('2'),word('3')],h,[0,2])

(S('1',SAttrs<86451656>(pre='',changers=set(),tags=set())), S('3',SAttrs<86452424>(pre='',changers=set(),tags=set())))


[(3, None)]

In [23]:
s=S('qwe')
s.attrs=8
s=='qwe'

True

In [24]:
s=S('qwe')
s.attrs

SAttrs<82776648>(pre='',changers=set(),tags=set())

In [25]:
s=S('qwe')
copy(s)

S('qwe',SAttrs<86309256>(pre='',changers=set(),tags=set()))

In [26]:
s=S('qwe')
repr(s)

"S('qwe',SAttrs<86309064>(pre='',changers=set(),tags=set()))"

# Словарь

## Классы отображения

### Базовые

In [27]:
def I(**args):
    if len(args)!=1:
        raise ValueError()
    p=[i for i in args.items()][0]
    if p[0] not in {'maindep','dep','nodep','punct','nomer','quantity',
                   'main','ip'}:
        raise ValueError()
    return p


In [28]:
class Struct:
    __slots__=['attrs','talk']
    def __init__(self,attrs=None):
        if attrs==None:
            self.attrs=SAttrs()
        elif type(attrs)==list:
#            self.attrs=SAttrs()
            self.attrs=SAttrs(pre=attrs[0][1].attrs.pre)
            attrs[0][1].attrs.pre=''
            pass # выделеие общих тегов
        else:
            self.attrs=attrs
        #self.talk=talk    #массив структур или туплов (строка-тип, ...)
    def __repr__(self):
        raise NotImplementedError('virtual function')
    def __str__(self):
        raise NotImplementedError('virtual function')
    def check_attrs(self,mes):
        if self.word==None:
            if self.talk[0][1].attrs.pre!='':
                print(repr(self))
            assert self.talk[0][1].attrs.pre=='', mes


In [29]:
class StC(Struct): # Container
# nodep
    def __init__(self,talk):
        assert type(talk)==list
        Struct.__init__(self,talk)
        self.talk=talk
        
    def __repr__(self):
        return 'StContainer<'+str(id(self))+'>('+\
            repr(self.talk) +')'#_'+repr(self.attrs)

    def __str__(self):
        return self.attrs.change( SAttrs.join(i[1] for i in self.talk) )
    

In [30]:
show_verb_map={}

class StVerb(Struct):
# main maindep ip rp dp vp tp pp gde kogda skolko
    __slots__=['word','oasp','asp','_form','_rod','_chis','_pers']
    def __init__(self,word,oasp=0,asp=None,form=None,rod=0,chis=0,pers=0):
        if type(word)==str:
            Struct.__init__(self)
            self.word=word
            assert oasp==None or type(oasp)==str ,\
                'oasp should be str or None'
            self.oasp=oasp
        else:
            Struct.__init__(self,word)# для тегов
            self.word=None
            self.talk=word

            found=None
            for i in word:
                if i[0]=='main' or i[0]=='maindep':
                    if found!=None:
                        raise ValueError('we must have only one main or maindep')
                    found =i[1]
                    if asp!=None or form!=None or chis!=0 or pers!=0:
                        raise ValueError('main or maindep may conflits with asp/form/pers')
                    asp=found.asp
                    form=found.form
                    rod=found.rod
                    chis=found.chis
                    pers=found.pers
        assert asp=='sov' or asp=='nesov' , 'asp should be "sov" or "nesov"'
        assert form=='neopr' or form=='povel' or form=='nast' ,\
            'form should be "neopr" or "povel" or "nast"'
        assert form=='neopr' and chis==None or \
                form!='neopr' and (chis=='ed' or chis=='mn'),\
            'wrong chis'
        assert (form=='neopr' or form=='povel') and rod==None or \
                not(form=='neopr' or form=='povel') and (rod in {'m','g','s'}),\
            'wrong rod'
        assert (form=='neopr' or form=='povel') and pers==None or \
                not(form=='neopr' or form=='povel') and (pers in {1,2,3}),\
            'wrong person'
        self.asp=asp
        self._form=form
        self._rod=rod
        self._chis=chis
        self._pers=pers
    
    form=property()
    @form.getter
    def form(self):
        return self._form
    @form.setter
    def form(self,val):
        self._form=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='dep' or i[0]=='maindep' :
                    i[1].form=val
                
    rod=property()
    @rod.getter
    def rod(self):
        return self._rod
    @rod.setter
    def rod(self,val):
        self._rod=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='dep' or i[0]=='maindep' :
                    i[1].rod=val
                
    chis=property()
    @chis.getter
    def chis(self):
        return self._chis
    @chis.setter
    def chis(self,val):
        self._chis=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='dep' or i[0]=='maindep' :
                    i[1].chis=val
                
    pers=property()
    @pers.getter
    def pers(self):
        return self._pers
    @pers.setter
    def pers(self,val):
        self._pers=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='dep' or i[0]=='maindep' :
                    i[1].pers=val
                
    def __repr__(self):
            #'<'+str(id(self))+'>'+\
        return 'StVerb'+\
            '('+(repr(self.talk) if self.word==None \
                 else repr(self.word)+','+repr(self.oasp))+','+\
            'asp='+repr(self.asp)+','+\
            'form='+repr(self.form)+','+\
            'chis='+repr(self.chis)+','+\
            'pers='+repr(self.pers)+')'

    def __str__(self):
        return self.attrs.change(
            SAttrs.join(i[1] for i in self.talk)
                if self.word==None else show_verb_map[self.word](self)
            )


### Склоняемые

In [31]:
class StDeclinable(Struct):
    __slots__=['word','odush','rod','chis','_pad']
    
    @staticmethod
    def odush_checker(odush):
        if type(odush)!=bool : raise TypeError('odush must be bool')
        return odush
        
    @staticmethod
    def rod_checker(rod):
        if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
        return rod

    @staticmethod
    def chis_checker(chis):
        if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
        return chis

    @staticmethod
    def pad_checker(pad):
        if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
            raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
        return pad

    def __init__(self,word,o=None,r=None,c=None,p=None):
        if type(word)==str:
            #Struct.__init__(self,word.attrs)
            # в словаре атрибуты отсутствуют
            # при парсинге узел копируется со словаря и туда добавляются атрибуты
            Struct.__init__(self)
            # но тем не менее все должны иметь атрибуты 
            # например для словосочетаний в словаре паттернов
            self.word=word
        elif type(word)==list:
            Struct.__init__(self,word)# для тегов
            self.word=None
            self.talk=word
            assert self.talk[0][1].attrs.pre==''
            
            found=None
            for i in word:
                if i[0]=='maindep':
                    if found!=None:
                        raise ValueError('we must have only one maindep')
                    found =i[1]
                    if o!=None or r!=None or c!=None:
                        raise ValueError('maindep may conflits with o/r/c')
                    o=found.odush
                    r=found.rod
                    c=found.chis
                    if p==None: p=found.pad
        self.odush=self.odush_checker(o)
        self.rod  =self.  rod_checker(r)
        self.chis =self. chis_checker(c)
        self._pad =self.  pad_checker(p)
        
    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val):
        self._pad=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='dep' or i[0]=='maindep' :
                    i[1].pad=val
                
    def post_repr(self):
        return 'o='+repr(self.odush)+',r='+repr(self.rod)+\
            ',c='+repr(self.chis)+',p='+repr(self.pad)+')'#_'+repr(self.attrs)
    def __repr__(self):
        raise NotImplementedError('virtual function')

    show_map=None
    def __str__(self):
        return self.attrs.change(
            SAttrs.join(i[1] for i in self.talk)
                if self.word==None else self.show_map[self.word](self)
            )


In [32]:
show_noun_map={}

# Существительное
class StNoun(StDeclinable):
# dep, maindep, nodep, nomer, punct
    __slots__=['_chis','och']
    def __init__(self,word=None,och=0,o=None,r=None,c=None,p=None):
        if type(word)==list:
            if och!=0: raise ValueError('Noun-och must be in leaf')
        else:
            if type(och)!=str and och!=None: raise ValueError('och must be str')
            self.och=och
        StDeclinable.__init__(self,word,o,r,c,p)
    
    chis=property()
    @chis.getter
    def chis(self):
        return self._chis
    @chis.setter
    def chis(self,val):
        #print('set chis')#,repr(self))
        if not hasattr(self,'_chis'):
            self._chis=val
        elif self._chis!=val:
            #print('change chis')
            self._chis=val
            if self.word!=None:
                if self.och==None :
                    pass
                elif type(self.och)==str:
                    tmp=self.och
                    self.och=self.word
                    self.word=tmp
                else:
                    raise RuntimeError('internal error: StNoun.och = '+self.och)
            else:
                for i in self.talk:
                    if i[0]=='dep' or i[0]=='maindep' :
                        i[1].chis=val

    pers=3
        
    def __repr__(self):
        return 'StNoun<'+str(id(self))+'>('+\
            (repr(self.talk) if self.word==None \
                 else repr(self.word)+','+repr(self.och))+','+\
            self.post_repr()
    
    show_map=show_noun_map

def show_noun1(st,word,ip,rp,dp,vp,tp,pp):
    if st.pad=='ip' : return word+ip
    if st.pad=='rp' : return word+rp
    if st.pad=='dp' : return word+dp
    if st.pad=='vp' : return word+vp
    if st.pad=='tp' : return word+tp
    if st.pad=='pp' : return word+pp
    raise RuntimeError()


In [33]:
# Местоимение личное
class StProNoun(StNoun):
    __slots__='pers'
    def __init__(self,word=None,och=0,pers=None,o=None,r=None,c=None,p=None):
        assert type(word)==str
        StNoun.__init__(self,word,och,o,r,c,p)
        assert pers in {1,2,3}
        self.pers=pers

In [34]:
show_num_map={}

# Числительное
class StNum(StDeclinable):
# maindep, quantity
    __slots__=['quantity']
    def __init__(self,word=None,quantity=None,o=None,r=None,c=None,p=None):
        if quantity!='1' and quantity!='2-4' and quantity!='>=5':
            raise TypeError('quantity must be "1", "2-4" or ">=5"')
        self.quantity=quantity
        StDeclinable.__init__(self,word,o,r,c,p)

    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val): #...
        #print('num.pad',val)
        self._pad=val
        if self.word==None:
            for i in self.talk:
                if i[0]=='quantity':
                    i[1].pad=val
                if i[0]=='dep' or i[0]=='maindep' :
                    if self.quantity=='1':
                        i[1].pad=val
                    elif self.quantity=='2-4':
                        i[1].chis='mn'
                        if val=='ip':
                            i[1].chis='ed'
                            i[1].pad='rp'
                        elif val=='vp':
                            if i[1].odush :
                                i[1].pad='rp'
                            else:
                                i[1].chis='ed'
                                i[1].pad='rp'
                        else:
                            i[1].pad=val
                    elif self.quantity=='>=5':
                        if val=='ip' or val=='vp':
                            i[1].pad='rp'
                        else:
                            i[1].pad=val
                    else:
                        raise RuntimeError()

    def __repr__(self):
        return 'StNum<'+str(id(self))+'>('+\
            (repr(self.talk) if self.word==None \
                 else repr(self.word))+','+\
            repr(self.quantity)+','+\
            self.post_repr()
    
    show_map=show_num_map

def show_num1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    else: raise RuntimeError()
        
    return word+rez


In [35]:
show_adj_map={}

# Прилагательное
class StAdj(StDeclinable):
#
    @staticmethod
    def pad_checker(pad):
        if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and \
                pad!='pp' and pad!='sh' : 
            raise TypeError('pad must be ip, rp, dp, vp, tp, pp or sh')
        return pad# пока особого смысла в этом нет
    
    def __init__(self,word=None,o=None,r=None,c=None,p=None):
        StDeclinable.__init__(self,word,o,r,c,p)

    def __repr__(self):
        return 'StAdj<'+str(id(self))+'>('+\
            (repr(self.talk) if self.word==None \
                 else repr(self.word))+','+\
            self.post_repr()
    
    show_map=show_adj_map
    
def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise RuntimeError()
        
    return word+rez


## Отображение

In [36]:
#show_noun_map['я']=lambda st: show_noun1(st,"","я","меня","мне","меня","мной","мне")


In [37]:
show_adj_map['летучий'] =\
    lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    )

In [38]:
show_num_map['один']=\
    lambda st: show_num1(st,'од' ,
        {
            'm' :('ин','ного','ному',('ного','ин'),'ним' ,'ном'),
            's' :('но','ного','ному',('но'  ,'но'),'ним' ,'ном'),
            'g' :('на','ной' ,'ной' ,('ну'  ,'ну'),'ной' ,'ной'),
            'mn':('ни','них' ,'ним' ,('них', 'ни'),'ними','них'),
        }
    )
show_num_map['два']=\
    (lambda st: \
        ("две" if st.rod=='g' else "два")                         if st.pad=='ip' else \
        "двух"                                                    if st.pad=='rp' else \
        "двум"                                                    if st.pad=='dp' else \
        ("двух" if st.odush else "две" if st.rod=='g' else "два") if st.pad=='vp' else \
        "двумя"                                                   if st.pad=='tp' else \
        "двух"                                                    if st.pad=='pp' else \
        throw(RuntimeError('unknown pad: '+st.pad))
    )
show_num_map['три']=\
    lambda st: show_num1(st,'тр' ,
        {   'mn':('и','ёх' ,'ём' ,('ёх', 'и'),'емя','ёх'), }    )
show_num_map['четыре']=\
    lambda st: show_num1(st,'четыр' ,
        {   'mn':('е','ёх' ,'ём' ,('ёх', 'е'),'ьмя','ёх'), }    )
show_num_map['пять']=\
    lambda st: show_num1(st,'пят' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )

def show_verb1(st,word,end_map):
    if st.form=='neopr' or st.form=='povel':
        end = end_map[st.form]
    else:
        (i,you,he,shi,it,we,yous,they)=end_map[st.form]
        if   (st.pers,st.chis)==(1,'ed'):
            end = i
        elif (st.pers,st.chis)==(2,'ed'):
            end = you
        elif (st.pers,st.chis)==(3,'ed'):
            if   st.rod=='m' : end=he
            elif st.rod=='g' : end=she
            elif st.rod=='s' : end=it
        elif (st.pers,st.chis)==(1,'mn'):
            end = we
        elif (st.pers,st.chis)==(2,'mn'):
            end = yous
        elif (st.pers,st.chis)==(3,'mn'):
            end = they
    return word+end

show_verb_map['видеть']=\
    lambda st: show_verb1(st,'ви',
        {    'neopr':"деть",
             'povel':"дь",
             'nast': ("жу","дишь","дит","дит","дит","дим","дите","дят")
        }   )

## Правила

### Словарь

In [39]:
ruwords={} #какие-то однословные правила складываются сюда, а какие-то остаются в функциях

def add_runoun(name,och,odush,rod,chis,word,ip,rp,dp,vp,tp,pp):
    ruwords[name]=StNoun(name,och,odush,rod,chis,'ip')
    show_noun_map[name]=lambda st: show_noun1(st,word,ip,rp,dp,vp,tp,pp)

def add_runoun2(name,rod,word,ip,rp,dp,vp,tp,pp,\
               mname,odush,mword,mip,mrp,mdp,mvp,mtp,mpp):
    add_runoun(name,mname,odush,rod,'ed',word,ip,rp,dp,vp,tp,pp)
    add_runoun(mname,name,odush,rod,'mn',mword,mip,mrp,mdp,mvp,mtp,mpp)


In [40]:
ruwords["я (муж)"]=StProNoun("я","мы",1,True,'m','ed','ip')
ruwords["я (жен)"]=StProNoun("я","мы",1,True,'g','ed','ip')
show_noun_map["я"]=lambda st: show_noun1(st,'',"я","меня","мне","меня","мной","мне")

#                                  ип ,нет ,дать,вижу,творю,думаю
add_runoun2("кот"   ,'m'  ,'кот'  ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "коты"  ,True ,'кот'  ,'ы','ов','ам','ов','ами','ах')

add_runoun2("джем"  ,'m'  ,'джем' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "джемы" ,False,'джем' ,'ы','ов','ам','ы' ,'ами','ах')

add_runoun2("ящик"  ,'m'  ,'ящик' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "ящики" ,False,'ящик' ,'и','ов','ам','и' ,'ами','ах')
add_runoun2("урок"  ,'m'  ,'урок' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "уроки" ,False,'урок' ,'и','ов','ам','и' ,'ами','ах')

add_runoun2("кошка" ,'g'  ,'кошк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кошки" ,True ,'кош'  ,'ки','ек','кам','ек','ками','ках')

add_runoun2("собака",'g'  ,'собак','а' ,'и' ,'е'  ,'у'  ,'ой' ,'е' ,
            "собаки",True ,'собак','и' ,''  ,'ам' ,''   ,'ами','ах')

add_runoun2("ручка" ,'g'  ,'ручк' ,'а' ,'и' ,'е'  ,'у'  ,'ой' ,'е'  ,
            "ручки" ,False,'руч'  ,'ки','ек','кам','ки','ками','ках')
add_runoun2("чашка" ,'g'  ,'чашк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "чашки" ,False,'чаш'  ,'ки','ек','кам','ки','ками','ках')

add_runoun2("кепка"  ,'g'  ,'кепк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кепки"  ,False,'кеп'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("шапка"  ,'g'  ,'шапк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "шапки"  ,False,'шап'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("коробка",'g'  ,'коробк','а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "коробки",False,'короб' ,'ки','ок','кам','ки','ками','ках')

add_runoun2("крыса" ,'g'  ,'крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "крысы" ,True ,'крыс' ,'ы','' ,'ам',''  ,'ами','ах')
add_runoun2("курица",'g'  ,'куриц','а','ы','е' ,'у' ,'ой' ,'е' ,
            "курицы",True ,'куриц','ы','' ,'ам',''  ,'ами','ах')
add_runoun2("лиса"  ,'g'  ,'лис'  ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "лисы"  ,True ,'лис'  ,'ы','' ,'ам',''  ,'ами','ах')
    
add_runoun2("шляпа" ,'g'  ,'шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "шляпы" ,False,'шляп' ,'ы','' ,'ам','ы' ,'ами','ах')

add_runoun2("свинья",'g'  ,'свинь','я' ,'и' ,'е'  ,'ю' ,'ёй'  ,'е'  ,
            "свиньи",True ,'свин' ,'ьи','ей','ьям','ей','ьями','ьях')

add_runoun2("мышь"   ,'g'  ,'мыш'   ,'ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "мыши"   ,True ,'мыш'   ,'и','ей','ам','ей','ами','ах')

add_runoun2("кровать",'g'  ,'кроват','ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "кровати",False,'кроват','и','ей','ям','и' ,'ями','ях')

add_runoun2("ружьё"  ,'s'  ,'ружь'  ,'ё' ,'я' ,'ю'  ,'ё' ,'ём'  ,'е'  ,
            "ружья"  ,False,'руж'   ,'ья','ей','ьям','ьи','ьями','ьях')

add_runoun2("вареньё",'s'  ,'варень','е' ,'я' ,'ю'  ,'е' ,'ем' ,'е' ,
            "варенья",False,'варен' ,'ья','ий','ьям','ья','ьями','ьях')

add_runoun2("ребёнок",'m'  ,'ребён','ок' ,'ка' ,'ку'  ,'ка' ,'ком' ,'ке' ,
            "дети"   ,True ,'дет' ,'и','ей','ям','ей','ьми','ях')

add_runoun('часы (предмет)',None,False,'m','mn','час','ы','ов','ам','ы','ами','ах')


In [41]:
def r_letuchij(): return StAdj('летучий',True,'g','ed','ip')

def r_odin(): return StNum('один','1',False,'m','ed','ip')
def r_dva(): return StNum('два','2-4',False,'m','mn','ip')
def r_tri(): return StNum('три','2-4',False,'m','mn','ip')
def r_chetyre(): return StNum('четыре','2-4',False,'m','mn','ip')
def r_pyat(): return StNum('пять','>=5',False,'m','mn','ip')

#(self,word,oasp=0,asp=None,form=None,rod=0,chis=0,pers=0)
def r_videt(): return StVerb('видеть',"увидеть",'nesov','neopr',None,None,None)

### Составные

In [42]:
def r_noun_and_noun(sn,a,n):
    return StNoun([
        I(dep=sn),
        I(nodep=S('и',a.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')
def r_noun_comma_noun(sn,c,n):
    return StNoun([
        I(dep=sn),
        I(punct=S(',',c.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')


In [43]:
def r_a_noun(a,n): 
    return SAttrs.to_right(a,n)

def r_adj_noun(a,n): 
    return StNoun([
        I(dep=a),
        I(maindep=n)
    ])


In [44]:
def r_numeral_noun(num,n):
    if num.chis!=n.chis :
        print('не совпадают числа числ. и сущ.:',str(num),str(n))
        num.chis=n.chis
    num.rod=n.rod
    num.odush=n.odush
    return StNum([
        I(quantity=num),
        I(maindep=n)
    ],quantity=num.quantity)

def r_noun_numeral(n,num):
    return StNoun([
        I(maindep=n),
        I(nomer=num)
    ])


In [45]:
def r_to_verb(t,v): 
    return SAttrs.to_right(t,v)

def r_noun_verb(n,v): 
    v.form='nast'
    v.pers=n.pers
    v.chis=n.chis
    v.rod=n.rod
    return StVerb([I(ip=n),I(main=v)])
def r_verb_noun(v,n): 
    n.pad='vp'
    return StVerb([I(maindep=v),I(ip=n)])


## Паттерны

### Словарь

In [46]:
dict_adj={
    'a' : S(''),
    #'a' : r_nekotoryj(),
    'an' : S(''),
    #'an' : r_nekotoryj(),
}

In [47]:
dict_noun={
    'I': ruwords["я (муж)"],
    #'I': ruwords["я (жен)"],
    
    #'cat':   ruwords["кошка"],
    'cat':   ruwords["кот"],
    #'cats':   ruwords["кошки"],
    'cats':   ruwords["коты"],
    
    #'rat':   ruwords["мышь"],
    'rat':   ruwords['крыса'],
    #'rats':   ruwords["мыши"],
    'rats':   ruwords['крысы'],
    
    'bat':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мышь"])),
    'bats':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мыши"])),
    
    'lesson':ruwords["урок"],
    'lessons':ruwords["уроки"],
    
    'cap':   ruwords["кепка"],
    #'cap':  ruwords["шапка"],
    'caps':   ruwords["кепки"],
    #'caps':  ruwords["шапки"],
    
    'pen':   ruwords["ручка"],
    'pens':   ruwords["ручки"],
    
    'hat':   ruwords["шляпа"],
    'hats':   ruwords["шляпы"],
    
    'hen':   ruwords["курица"],
    'hens':   ruwords["курицы"],
    
    'dog':   ruwords['собака'],
    'dogs':   ruwords['собаки'],
    'pig':   ruwords['свинья'],
    'pigs':   ruwords['свиньи'],
    'gun':   ruwords['ружьё'],
    'guns':   ruwords['ружья'],
    'cup':   ruwords['чашка'],
    'cups':   ruwords['чашки'],
    
    #'box':   ruwords['коробка'],
    #'boxes':   ruwords['коробки'],
    'box':   ruwords['ящик'],
    'boxes':   ruwords['ящики'],

    'jam':   ruwords['джем'],
    #'jam':   ruwords['варенье'],

    'bed':   ruwords['кровать'],
    'beds':   ruwords['кровати'],

    'fox':   ruwords['лиса'],
    'foxes':   ruwords['лисы'],
    
    'child': ruwords['ребёнок'],
    'children': ruwords['дети'],
    
    'watch': ruwords['часы (предмет)']
}

In [48]:
dict_numeral={
    'one':   r_odin(),
    'two':   r_dva(),
    'three': r_tri(),
    'four':  r_chetyre(),
    'five':  r_pyat(),
}

In [49]:
dict_verb={
    'see': r_videt(),
}

### Составные

In [50]:
def debug_pp(fun):
    def wrapper(s,p):
        print('{'+'.'*p+fun.__name__)
        rezs=fun(s,p)
        print('_'+'.'*p+str(len(rezs)),'in ',fun.__name__,'}',
              [(p,str(r)) for (p,r) in rezs],'\n')
        for i in rezs:
            if isinstance(i[1],StDeclinable):
                i[1].check_attrs('wrapper:'+fun.__name__)
        return rezs
#    return wrapper
    return fun

In [51]:
@debug_pp
def p_numeral(s,p):
    return fromdict(dict_numeral)(s,p)

In [52]:
#2->
@debug_pp
def p_adj(s,p):
    return fromdict(dict_adj)(s,p)

In [53]:
@debug_pp
def p_adj_noun3(s,p):
    rezs=[]
    if 1: #исключение словосочетания
        def lp_an_a(str,pos):
            lrezs=word('an')(str,pos)
            if len(lrezs)>0 : return lrezs
            return word('a')(str,pos)
        rezs+=p_seq(s,p,[ lp_an_a, p_noun3 ],r_a_noun)
        #return p_seq(s,p,[ lp_an_a, p_noun3 ],r_nekotoryj_noun)
    if len(rezs)>0 : return rezs
    #словосочетание
    return p_seq(s,p,[ p_adj, p_noun3 ],r_adj_noun)    
    
@debug_pp
def p_noun3(s,p):
    rezs=  p_adj_noun3(s,p)
    rezs+= p_numeral(s,p)
    rezs+= fromdict(dict_noun)(s,p)
    return rezs

@debug_pp
def p_noun2(s,p):
    rezs=  p_seq(s,p,[ p_noun3, p_numeral ], r_noun_numeral)
    rezs+= p_noun3(s,p)
    return rezs

@debug_pp
def p_noun1(s,p):
    rezs=  p_seq(s,p,[ p_numeral, p_noun2 ], r_numeral_noun)
    rezs+= p_noun2(s,p)
    return rezs

#1->
@debug_pp
def p_noun(s,p):
    @debug_pp
    def p_noun1_and_noun(s,p):
        rezs=  p_seq(s,p,[ p_noun1, word('and'), p_noun ],r_noun_and_noun  )
        rezs+= p_seq(s,p,[ p_noun1, word(',')  , p_noun ],r_noun_comma_noun)
        return rezs
    rezs=  p_noun1_and_noun(s,p)
    rezs+= p_noun1(s,p)
    return rezs

In [54]:
@debug_pp
def p_verb1(s,p):
    rezs=  p_seq(s,p,[fromdict(dict_verb),p_noun],r_verb_noun)
#    rezs+= p_seq(s,p,[word('to'),fromdict(dict_verb)],r_to_verb)
    if len(rezs)>0: return rezs
    rezs+= fromdict(dict_verb)(s,p)
    return rezs

@debug_pp
def p_verb(s,p):
    rezs=  p_seq(s,p,[p_noun,p_verb1],r_noun_verb)
    rezs+= p_seq(s,p,[word('to'),p_verb1],r_to_verb)
    if len(rezs)>0: return rezs
    rezs+= p_verb1(s,p)
    return rezs

In [55]:
@debug_pp
def p_phrase(s,p):
    rezs=p_verb(s,p)
    if len(rezs)>0: return rezs
    rezs=p_noun(s,p)
    if len(rezs)>0: return rezs
    rezs=p_adj(s,p)
    return rezs

In [56]:
dict_proper={}
@debug_pp
def p_sentence(s,p):
    first_capital = s[p]=='I' or ch_title in s[p].attrs.changers
    def r_sentence(ph,d):
        rez=StC([I(nodep=ph),I(punct=d)])
        if first_capital: rez.attrs.changers|={ch_sentence}
        return rez
    restore_title=False
    if ch_title in s[p].attrs.changers and s[p] not in dict_proper:
        s[p].attrs.changers-={ch_title}
        restore_title=True
    rezs=p_seq(s,p,[p_phrase,word('.')],r_sentence)
    if restore_title:
        s[p].attrs.changers|={ch_title}
    return rezs

In [57]:
def en2ru(s): # main
    def maxlen_rezs(rezs):
        m=0
        im=set()
        for i in range(len(rezs)):
            if rezs[i][0]>m:
                m=rezs[i][0]
                im={i}
            elif rezs[i][0]==m:
                im.add(i)
        return [rezs[i] for i in im]
                
    s=[ i for i in tokenizer(s)]
    if len(s)==0:
        print('no tokens')
        return ''

    p=0
    rez=[]
    while p<len(s):
        rezs=maxlen_rezs(p_sentence(s,p))
        if len(rezs)==0: break
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        rez.append(I(nodep=r1))

    if len(rez)>0:
        if p!=len(s):
            print('NOT PARSED:')
            print(SAttrs.join(s[p:]))
        return str(StC(rez))
    else:
        assert p==0
        rezs=maxlen_rezs(p_phrase(s,p))
        if len(rezs)==0:
            raise ParseError('no results')
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        if p!=len(s):
            print('NOT PARSED!')
            print(SAttrs.join(s[p:]))
        return str(r1)


# Тесты

```
... изучить модули
выделить в модули общее и классы
сделать ф-цию для тестирования
... изучить подключение к существующему ядру
и сделать отдельные ноутбуки с тестами,
иначе выделить модули тесты общего и тесты 
сделать переменную для дебага паттернов
? попробовать сделать кэширование результатов
    статистика использования паттернов/словарей/правил
попробовать сделать mod(x,prop=val,prop=val...)

местоимения вп
разбиение на dict_noun_uno & dict_noun_many
dict_verb, dict_verb_s
    7)  у меня есть
        у меня нет
        покажи
        дай
    8)  скажи
    9)  (находится) на/в
    10) вопросы-ответы
        считать от до
        ловить
    11) сколько
    13) где
    14) какого цвета
        КОНТЕКСТ
    15) посмотри

watch, двое, трое, пятеро
...
нужен LR-парсер, причем недетерминированный - посмотреть, какие есть библиотеки
    возможно стоит придумать уровни для паттернов...
атрибуты слов: (теги)
```

In [58]:
def decline1(s,pads=['ip','rp','dp','vp','tp','pp']):
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)!=1:
        raise TextError(rezs)
    tmp=rezs[0]
    
    m=[]
    for p in pads:
        #print(str(tmp))
        prompt= \
            '' if p=='ip' else\
            'нет ' if p=='rp' else\
            'дать ' if p=='dp' else\
            'вижу ' if p=='vp' else\
            'творю ' if p=='tp' else\
            'думаю о ' if p=='pp' else\
            throw(ValueError('bad pad: '+p))
        #rez=deepcopy(tmp)
        tmp.pad=p
        m.append(prompt+str(tmp))#        print(prompt+str(tmp))
    return m

def decline(s,pads=['ip','rp','dp','vp','tp','pp']):
    for i in decline1(s,pads):
        print(i)
    
def decline_test(s,rezs,pads=['ip','rp','dp','vp','tp','pp']):
    tmp=decline1(s,pads)
    if tmp==rezs:
        print('OK')
    else:
        print('TestError')
        if len(tmp)!=len(rezs):
            print(tmp)
        else:
            for i in range(len(tmp)):
                if tmp[i]!=rezs[i]:
                    print(i+1,')',tmp[i])
    

In [59]:
decline('one watch')

не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы
одни часы
нет одних часов
дать одним часам
вижу одни часы
творю одними часами
думаю о одних часах


## Lesson 1

In [60]:
en2ru('cat')

'кот'

In [61]:
en2ru('a cat')

'кот'

In [62]:
en2ru('rat')

'крыса'

In [63]:
en2ru('bat')

'летучая мышь'

In [64]:
en2ru('a')

''

In [65]:
en2ru('lesson')

'урок'

In [66]:
en2ru('_Cat and cat')

'_Кот и кот'

In [67]:
en2ru('_a cat and cat')

'_кот и кот'

In [68]:
en2ru('_a cat , cat')

'_кот, кот'

In [69]:
x=p_noun(tokenize('_a cat, cat'),0)[0][1]
str(x)

'_кот, кот'

In [70]:
s=tokenize('_a cat, cat')
rezs=p_seq(s,0,[ p_noun1, word(',')  , p_noun ],r_noun_comma_noun)
str(rezs[0][1])

'_кот, кот'

In [71]:
str(p_noun(tokenize('cat,_cat,_a cat'),0)[0][1])

'кот,_кот,_кот'

In [72]:
SAttrs.join(tokenize('cat,_cat,_a cat'))

'cat,_cat,_a cat'

In [73]:
en2ru('rat, cat')

'крыса, кот'

## Lesson 2

In [74]:
en2ru('cat AND_Rat')

'кот И_Крыса'

In [75]:
en2ru('a pen and a hen')

'ручка и курица'

In [76]:
en2ru('a bat and a cat')

'летучая мышь и кот'

In [77]:
en2ru('a hen, a bat, a cat and a rat')

'курица, летучая мышь, кот и крыса'

## Lesson 3

In [78]:
en2ru('one')

'один'

In [79]:
en2ru('a bat, a dog and a pig')

'летучая мышь, собака и свинья'

In [80]:
en2ru('one dog')

'одна собака'

In [81]:
en2ru('three dogs')

'три собаки'

In [82]:
decline('rat and cat')

крыса и кот
нет крысы и кота
дать крысе и коту
вижу крысу и кота
творю крысой и котом
думаю о крысе и коте


In [83]:
decline('one bat and three cups')

одна летучая мышь и три чашки
нет одной летучей мыши и трёх чашек
дать одной летучей мыши и трём чашкам
вижу одну летучую мышь и три чашки
творю одной летучей мышью и тремя чашками
думаю о одной летучей мыши и трёх чашках


In [84]:
en2ru('one bat and three cups')

'одна летучая мышь и три чашки'

In [85]:
decline_test('one, two, three, four, five',[
    'один, два, три, четыре, пять',
    'нет одного, двух, трёх, четырёх, пяти',
    'дать одному, двум, трём, четырём, пяти',
    'вижу один, два, три, четыре, пять',
    'творю одним, двумя, тремя, четырьмя, пятью',
    'думаю о одном, двух, трёх, четырёх, пяти',])

OK


In [86]:
decline_test('cat, one cat, cats, two cats, three cats, five cats',[
    'кот, один кот, коты, два кота, три кота, пять котов',
    'нет кота, одного кота, котов, двух котов, трёх котов, пяти котов',
    'дать коту, одному коту, котам, двум котам, трём котам, пяти котам',
    'вижу кота, одного кота, котов, двух котов, трёх котов, пять котов',
    'творю котом, одним котом, котами, двумя котами, тремя котами, пятью котами',
    'думаю о коте, одном коте, котах, двух котах, трёх котах, пяти котах',
])

OK


In [87]:
decline_test('lesson, one lesson, lessons, two lessons, three lessons, five lessons',[
    'урок, один урок, уроки, два урока, три урока, пять уроков',
    'нет урока, одного урока, уроков, двух уроков, трёх уроков, пяти уроков',
    'дать уроку, одному уроку, урокам, двум урокам, трём урокам, пяти урокам',
    'вижу урок, один урок, уроки, два урока, три урока, пять уроков',
    'творю уроком, одним уроком, уроками, двумя уроками, тремя уроками, пятью уроками',
    'думаю о уроке, одном уроке, уроках, двух уроках, трёх уроках, пяти уроках',
])

OK


In [88]:
decline_test('rat, one rat, rats, two rats, three rats, five rats',[
    'крыса, одна крыса, крысы, две крысы, три крысы, пять крыс',
    'нет крысы, одной крысы, крыс, двух крыс, трёх крыс, пяти крыс',
    'дать крысе, одной крысе, крысам, двум крысам, трём крысам, пяти крысам',
    'вижу крысу, одну крысу, крыс, двух крыс, трёх крыс, пять крыс',
    'творю крысой, одной крысой, крысами, двумя крысами, тремя крысами, пятью крысами',
    'думаю о крысе, одной крысе, крысах, двух крысах, трёх крысах, пяти крысах',
])

OK


In [89]:
decline('child, one child, children, two children, three children, five children')

ребёнок, один ребёнок, дети, два ребёнка, три ребёнка, пять детей
нет ребёнка, одного ребёнка, детей, двух детей, трёх детей, пяти детей
дать ребёнку, одному ребёнку, детям, двум детям, трём детям, пяти детям
вижу ребёнка, одного ребёнка, детей, двух детей, трёх детей, пять детей
творю ребёнком, одним ребёнком, детьми, двумя детьми, тремя детьми, пятью детьми
думаю о ребёнке, одном ребёнке, детях, двух детях, трёх детях, пяти детях


## Lesson 4

In [90]:
print(en2ru(
'''  A hat, a cup and a box.
A bat, a hen and a fox.'''
))

  Шляпа, чашка и ящик.
Летучая мышь, курица и лиса.


In [91]:
en2ru('I')

'я'

In [92]:
en2ru('to see')

'видеть'

In [93]:
en2ru('I see')

'я вижу'

In [94]:
en2ru('see cat')

'видеть кота'

In [99]:
print(en2ru('''I see
  a pig and a hen,
  a dog and a gun,
  a cat and a hat.'''))

Я вижу
  свинью и курицу,
  собаку и ружьё,
  кота и шляпу.


In [96]:
print(en2ru('''I see one box, one hat and one cap'''))

я вижу один ящик, одну шляпу и одну кепку


In [97]:
print(en2ru('''I see jam and one cup'''))

я вижу джем и одну чашку


## Lesson 5

In [98]:
en2ru('')

no tokens


''